In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [9]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
preprocessing.generate_spectogram_images(params)
preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

Window size is bigger than record segment: specgram_matrix_S-acelera11_segment24.png
Window size is bigger than record segment: specgram_matrix_S-acelera4_segment0.png
Window size is bigger than record segment: specgram_matrix_S-acelera5_segment11.png
Window size is bigger than record segment: specgram_matrix_S-acelera6_segment15.png
Window size is bigger than record segment: specgram_matrix_S-acelera7_segment16.png
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment0.png
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment1.png
Window size is bigger than record segment: specgram_matrix_S-arranca8_segment0.png
Window size is bigger than record segment: specgram_matrix_S-avanza2_segment0.png
Window size is bigger than record segment: specgram_matrix_S-avanza9_segment22.png
Window size is bigger than record segment: specgram_matrix_S-frena10_segment24.png
Window size is bigger than record segment: specgram_matrix_S-frena5_segment28.png
W

## Segund Paso: instanciar el modelo

In [8]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 55, 240, 64)       9472      
_________________________________________________________________
batch_normalization_4 (Batch (None, 55, 240, 64)       256       
_________________________________________________________________
activation_5 (Activation)    (None, 55, 240, 64)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 48, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 33792)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4325504   
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
__________

## Tercer paso: crear el generador

In [4]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2960 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [5]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(lr=0.001)

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

Epoch 1/25
296/296 [==============================] - 20s 66ms/step - loss: 0.5068 - sparse_categorical_accuracy: 0.8247 - val_loss: 0.2439 - val_sparse_categorical_accuracy: 0.9667oss: 0.5638 - sparse_categ - ETA: 1s - loss: 0.5341 - sparse_cate
Epoch 2/25
296/296 [==============================] - 18s 61ms/step - loss: 0.1164 - sparse_categorical_accuracy: 0.9652 - val_loss: 0.1839 - val_sparse_categorical_accuracy: 0.9000
Epoch 3/25
296/296 [==============================] - 18s 61ms/step - loss: 0.0848 - sparse_categorical_accuracy: 0.9666 - val_loss: 0.1344 - val_sparse_categorical_accuracy: 0.9667
Epoch 4/25
296/296 [==============================] - 18s 62ms/step - loss: 0.0393 - sparse_categorical_accuracy: 0.9889 - val_loss: 0.2825 - val_sparse_categorical_accuracy: 0.8667
Epoch 5/25
294/296 [============================>.] - ETA: 0s - loss: 0.0415 - sparse_categorical_accuracy: 0.9874- ETA: 7s - loss: 0.0401 - sparse_categoric - ETA: 2s - loss: 0.0398 - sparse_categorical_acc